In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import arff
import os

In [2]:
# Load the data.

dir_path = os.path.dirname(os.getcwd())
freq_data_path = os.path.join(dir_path, 'data/freMTPL2freq.arff')
sev_data_path = os.path.join(dir_path, 'data/freMTPL2sev.arff')

with open(freq_data_path) as f:
    freq_data = arff.load(f)
freq_df = pd.DataFrame(data=freq_data['data'], columns=[a[0] for a in freq_data['attributes']])

with open(sev_data_path) as f:
    sev_data = arff.load(f)
sev_df = pd.DataFrame(data=sev_data['data'], columns=[a[0] for a in sev_data['attributes']])

# Merge the frequency and severity data. 
df = pd.merge(freq_df, sev_df, on='IDpol', how='left')

# Convert the byte columns to categories.
for col in df.select_dtypes([object]):
    df[col] = df[col].apply(lambda x: x.decode() if isinstance(x, bytes) else x)
    df[col] = df[col].astype('category')

# Show the first rows.
df.head()

,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimAmount
0,1.0,1.0,0.10,D,5.0,0.0,55.0,50.0,B12,Regular,1217.0,R82,NaN
1,3.0,1.0,0.77,D,5.0,0.0,55.0,50.0,B12,Regular,1217.0,R82,NaN
2,5.0,1.0,0.75,B,6.0,2.0,52.0,50.0,B12,Diesel,54.0,R22,NaN
3,10.0,1.0,0.09,B,7.0,0.0,46.0,50.0,B12,Diesel,76.0,R72,NaN
4,11.0,1.0,0.84,B,7.0,0.0,46.0,50.0,B12,Diesel,76.0,R72,NaN


In [ ]:
# Check if there are missing values.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679513 entries, 0 to 679512
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   IDpol        679513 non-null  float64 
 1   ClaimNb      679513 non-null  float64 
 2   Exposure     679513 non-null  float64 
 3   Area         679513 non-null  category
 4   VehPower     679513 non-null  float64 
 5   VehAge       679513 non-null  float64 
 6   DrivAge      679513 non-null  float64 
 7   BonusMalus   679513 non-null  float64 
 8   VehBrand     679513 non-null  category
 9   VehGas       679513 non-null  category
 10  Density      679513 non-null  float64 
 11  Region       679513 non-null  category
 12  ClaimAmount  26444 non-null   float64 
dtypes: category(4), float64(9)
memory usage: 49.3 MB


In [7]:
df = df.rename(columns={
    'ClaimNb': 'n_claims',
    'Exposure': 'exposure',
    'DrivAge': 'driver_age',
    'VehAge': 'vehicle_age',
    'BonusMalus': 'bonus_malus',
    'Density': 'density',
    'Area': 'area',
    'VehPower': 'vehicle_power',
    'VehBrand': 'vehicle_brand',
    'VehGas': 'vehicle_gas',
    'Region': 'region',
    'ClaimAmount': 'claim_amount'
    })

In [8]:
df.columns

Index(['IDpol', 'n_claims', 'exposure', 'area', 'vehicle_power', 'vehicle_age',
       'driver_age', 'bonus_malus', 'vehicle_brand', 'vehicle_gas', 'density',
       'region', 'claim_amount'],
      dtype='object')

In [25]:
# Sanity checks. We can see other features' validity from OpenML
print(((df['driver_age']<18) | (df['driver_age'] > 100)).any(),
    (df['density']<0).any(),
    (df['exposure'] <= 0).any()
    )
     

False False False


In [26]:
df['frequency'] = df['n_claims'] / df['exposure']


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678013 entries, 0 to 678012
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   IDpol          678013 non-null  float64 
 1   n_claims       678013 non-null  float64 
 2   exposure       678013 non-null  float64 
 3   area           678013 non-null  category
 4   vehicle_power  678013 non-null  float64 
 5   vehicle_age    678013 non-null  float64 
 6   driver_age     678013 non-null  float64 
 7   bonus_malus    678013 non-null  float64 
 8   vehicle_brand  678013 non-null  category
 9   vehicle_gas    678013 non-null  category
 10  density        678013 non-null  float64 
 11  region         678013 non-null  category
 12  frequency      678013 non-null  float64 
dtypes: category(4), float64(9)
memory usage: 49.1 MB
